In [39]:
import io
import re
import random
import itertools

In [40]:
f = io.open("../dataset/curses_55k.txt", mode="r", encoding="utf-8")
data = f.read()
data = data.split('\n')

In [9]:
# Strips whitespaces from data
#data = [x.strip(' ') for x in data]

In [41]:
data[:10]

['gówn0',
 'afa',
 'bladź',
 'bl0łdżob',
 'brandzl0wać',
 'brandzl0wanie',
 'burdelchata',
 'burdelówa',
 'bzykać',
 'bzykank0']

In [42]:
# Create hash table with single letters as hashes
profanity_hash = {}
for word in data[:-1]:
    key = word[0]
    if key in profanity_hash.keys():
        profanity_hash[key].append(word)
    else:
        profanity_hash[key]=[word]

In [43]:
def censor(sentence = ""):
    new_sentence = ""
    
    for word in sentence.split():
        if word[0] in profanity_hash.keys():
            if word in profanity_hash[word[0]]:
                new_sentence += '*'*len(word) + ' '
            else:
                new_sentence += word + ' '
        else:
            new_sentence += word + ' '    
            
    return new_sentence[:-1] # [:-1] deletes space at the end

In [44]:
censor("Ja pierdolę co za chuj debil i to pisał a a a.")

'Ja ******** co za **** debil i to pisał a a a.'

In [45]:
def word_variances_alg(word):
    
    # changing polish chars
    word = word.replace('ą','a').replace('ę', 'e').replace('ź','z').replace('ż', 'z').replace('ó','o').replace('ł', 'l').replace('ć','c').replace('ń','n').replace('ś','s')
    
    dictionary_no_big_letters = {'o':'0','b':'8','s':'5','e':'3','g':'9','z':'2','a':'4','u':'v','w':'vv','l':'I','I':'l'}
    dictionary = {'o':'0','O':'0','b':'8','B':'8','s':'5','S':'5','e':'3','E':'3','G':'6','g':'9','z':'2','Z':'2','a':'@','A':'4','T':'7','u':'v','w':'vv','l':'I','I':'l'}

    list_keys = list(dictionary_no_big_letters.keys())
    list_values = list(dictionary_no_big_letters.values())
    word_variances = []
    lista = []
    output= []
    length = 20
    for i in range(length):
        lista.append(word)
    for key,value in dictionary_no_big_letters.items(): #loop which creating words with changed letters one by one from the first letter to the last letter
        for j in range(0,10):
             for i in range(0,length):
                    output = re.sub(key, value, lista[i], j)
                    word_variances.append(output)
    for key,value in dictionary_no_big_letters.items(): #loop which creating  words with changed letters one by one from the last letter to the first letter
        for j in range(0,10):
             for i in range(0,length):
                    regex = key + "(?!.*" + key + ")" 
                    output2 = re.sub(regex, value, lista[i], j)
                    word_variances.append(output2)
                       
    full_list = []
    mapIndexPosition = list(zip(list_keys, list_values))
    random.shuffle(mapIndexPosition)
    list_keys_1,list_values_1 = zip(*mapIndexPosition) #shuffeling of lists of keys and values(with no missing indexes) to make combination of two letters 
    
    mapIndexPosition2 = list(zip(list_keys, list_values))
    random.shuffle(mapIndexPosition2)
    list_keys_2,list_values_2 = zip(*mapIndexPosition2)

    for elem in range(len(list_keys)):
        list_out = [element.replace(list_keys[elem],list_values[elem]).replace(list_keys_1[elem],list_values_1[elem]).replace(list_keys_2[elem],list_values_2[elem]) for element in lista]
        full_list += list_out
    
    word_variances = word_variances + full_list #filling a list with variances

    for elem in range(len(list_keys_1)):
        list_out = [element.replace(list_keys_1[elem],list_values_1[elem]).replace(list_keys_2[elem],list_values_2[elem]) for element in lista]
        full_list += list_out
    
    word_variances = word_variances + full_list
    
    mapIndexPosition = list(zip(list_keys, list_values))
    random.shuffle(mapIndexPosition)
    list_keys_1,list_values_1 = zip(*mapIndexPosition)
    
    mapIndexPosition2 = list(zip(list_keys, list_values))
    random.shuffle(mapIndexPosition2)
    list_keys_2,list_values_2 = zip(*mapIndexPosition2)

    for elem in range(len(list_keys_1)):
        list_out = [element.replace(list_keys_1[elem],list_values_1[elem]).replace(list_keys_2[elem],list_values_2[elem]) for element in lista]
        full_list += list_out
    
    word_variances = word_variances + full_list
    
    for elem in range(len(list_keys)): # filling variances with 
        list_out = [element.replace(list_keys[0],list_values[0]).replace(list_keys[1],list_values[1]) for element in lista]
        full_list += list_out
    word_variances = word_variances + full_list
    
    for elem in range(len(list_keys)):
        list_out = [element.replace(list_keys[0],list_values[0]).replace(list_keys[6],list_values[6]) for element in lista]
        full_list += list_out
    word_variances = word_variances + full_list
    
    for elem in range(len(list_keys)):
        list_out = [element.replace(list_keys[3],list_values[3]).replace(list_keys[6],list_values[6]) for element in lista]
        full_list += list_out
    word_variances = word_variances + full_list
    
    for elem in range(len(list_keys)):
        list_out = [element.replace(list_keys[3],list_values[3]).replace(list_keys[6],list_values[6]).replace(list_keys[0],list_values[0]) for element in lista]
        full_list += list_out
    word_variances = word_variances + full_list
    
    for i in range(len(word)): #loop which generates words with space gaps every letter
        full_list.append(word[0:i]+" "+word[i:len(word)])
    word_variances = word_variances + full_list    
    for i in range(len(word)): #loop which generates words with double letter
        full_list.append(word[0:i]+word[i-1]+word[i:len(word)])
    word_variances = word_variances + full_list

    return list(set(word_variances))

In [46]:
def add_words(word: str, profanity_hash: dict) -> dict:
    word = word.lower()
    similar_words = word_variances_alg(word)
    print(f"similiar: {similar_words}")
    for word in similar_words:
        key = word[0]
        print(f"word={word}, key={key}")
        if key in profanity_hash.keys():
            if word not in profanity_hash[key]:
                print(f"dodaj {word}")
                profanity_hash[key].append(word)
        else:
            profanity_hash[key] = [word]

    return profanity_hash

In [47]:
changed_table = add_words("Jebać", profanity_hash)
# changed_table["c"]

similiar: ['jebac', 'jebaac', 'jjebac', 'jeba c', 'j3b4c', 'jeb ac', 'jeb4c', ' jebac', 'jeebac', 'je bac', 'j ebac', 'j3bac', 'je8ac', 'je84c', 'cjebac', 'jebbac', 'j38ac']
word=jebac, key=j
dodaj jebac
word=jebaac, key=j
dodaj jebaac
word=jjebac, key=j
dodaj jjebac
word=jeba c, key=j
dodaj jeba c
word=j3b4c, key=j
dodaj j3b4c
word=jeb ac, key=j
dodaj jeb ac
word=jeb4c, key=j
dodaj jeb4c
word= jebac, key= 
word=jeebac, key=j
dodaj jeebac
word=je bac, key=j
dodaj je bac
word=j ebac, key=j
dodaj j ebac
word=j3bac, key=j
dodaj j3bac
word=je8ac, key=j
dodaj je8ac
word=je84c, key=j
dodaj je84c
word=cjebac, key=c
dodaj cjebac
word=jebbac, key=j
dodaj jebbac
word=j38ac, key=j
dodaj j38ac
